In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [117]:
Dataset_dir = ["./train.csv", "./store.csv","./test.csv"]

In [118]:
def load_data(dir):
    data = pd.read_csv(dir[0],parse_dates=[2])
    store = pd.read_csv(dir[1])
    test = pd.read_csv(dir[2],parse_dates=[3])
    return data,store,test

In [119]:
train,store,test = load_data(Dataset_dir)
train = train[train['Open'] != 0]

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [120]:
train = train.merge(store, on='Store')
test = test.merge(store, on='Store')
print(train.columns)
mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
train.StoreType.replace(mappings, inplace=True)
enc = OneHotEncoder(sparse = False)
OH = enc.fit_transform(train[["StoreType"]]).flatten()
train["StoreType"] = pd.Series(OH)
print(train["StoreType"])


Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


0         0.0
1         0.0
2         1.0
3         0.0
4         0.0
5         0.0
6         1.0
7         0.0
8         0.0
9         0.0
10        1.0
11        0.0
12        0.0
13        0.0
14        1.0
15        0.0
16        0.0
17        0.0
18        1.0
19        0.0
20        0.0
21        0.0
22        1.0
23        0.0
24        0.0
25        0.0
26        1.0
27        0.0
28        0.0
29        0.0
         ... 
844362    0.0
844363    1.0
844364    0.0
844365    0.0
844366    0.0
844367    1.0
844368    0.0
844369    0.0
844370    0.0
844371    1.0
844372    0.0
844373    0.0
844374    0.0
844375    1.0
844376    0.0
844377    0.0
844378    0.0
844379    1.0
844380    0.0
844381    0.0
844382    0.0
844383    1.0
844384    0.0
844385    0.0
844386    0.0
844387    1.0
844388    0.0
844389    0.0
844390    0.0
844391    1.0
Name: StoreType, Length: 844392, dtype: float64


In [121]:
def build_feature(data,feature):
    data['Year'] = data.Date.dt.year
    data['Month'] = data.Date.dt.month
    data['Day'] = data.Date.dt.day
    